In [ ]:
# ====================================================
# STEP 1: Import Libraries
# ====================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import (
    roc_auc_score, classification_report, confusion_matrix, 
    roc_curve, precision_recall_curve, average_precision_score,
    accuracy_score, precision_score, recall_score, f1_score
)
import lightgbm as lgb
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings('ignore')

print("LightGBM Version:", lgb.__version__)
print("✅ Libraries imported successfully")

In [ ]:
# ====================================================
# STEP 2: Load Data
# ====================================================

train_transaction = pd.read_csv(f"./train_transaction.csv")
test_transaction  = pd.read_csv(f"./test_transaction.csv")

print(f"Train shape: {train_transaction.shape}")
print(f"Test shape: {test_transaction.shape}")
print(f"\nTrain dataset info:")
print(f"Columns: {train_transaction.shape[1]}")
print(f"Rows: {train_transaction.shape[0]}")

In [ ]:
# ====================================================
# STEP 3: Data Preprocessing
# ====================================================

# Separate features and target
X = train_transaction.drop(['isFraud', 'TransactionID'], axis=1)
y = train_transaction['isFraud']
X_test_final = test_transaction.drop(['TransactionID'], axis=1)
test_ids = test_transaction['TransactionID']

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"Test features shape: {X_test_final.shape}")

# Identify column types
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

print(f"\nNumerical columns: {len(numerical_cols)}")
print(f"Categorical columns: {len(categorical_cols)}")

In [ ]:
# ====================================================
# STEP 4: Handle Missing Values
# ====================================================

# Check missing values percentage
missing_pct = (X.isnull().sum() / len(X) * 100).sort_values(ascending=False)
print("Top 20 columns with highest missing values (%)")
print(missing_pct.head(20))

# Drop columns with >90% missing values
high_missing_cols = missing_pct[missing_pct > 90].index.tolist()
print(f"\nDropping {len(high_missing_cols)} columns with >90% missing values")

X = X.drop(columns=high_missing_cols)
X_test_final = X_test_final.drop(columns=high_missing_cols)

print(f"New features shape: {X.shape}")

In [ ]:
# ====================================================
# STEP 5: Label Encoding for Categorical Columns
# ====================================================

# Update categorical columns list after dropping
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
print(f"Categorical columns to encode: {categorical_cols}")

# Label Encoding
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    # Combine train and test for consistent encoding
    combined = pd.concat([X[col].astype(str), X_test_final[col].astype(str)])
    le.fit(combined)
    
    X[col] = le.transform(X[col].astype(str))
    X_test_final[col] = le.transform(X_test_final[col].astype(str))
    label_encoders[col] = le

print(f"\n✅ Encoded {len(categorical_cols)} categorical columns")
print(f"Final X shape: {X.shape}")
print(f"Final X_test shape: {X_test_final.shape}")

In [ ]:
# ====================================================
# STEP 6: Fill Remaining Missing Values
# ====================================================

# Fill numerical missing values with median
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
for col in numerical_cols:
    if X[col].isnull().sum() > 0:
        median_val = X[col].median()
        X[col] = X[col].fillna(median_val)
        X_test_final[col] = X_test_final[col].fillna(median_val)

print(f"Missing values in X after filling: {X.isnull().sum().sum()}")
print(f"Missing values in X_test after filling: {X_test_final.isnull().sum().sum()}")
print("✅ Data preprocessing completed")

In [ ]:
# ====================================================
# STEP 7: Train-Validation Split
# ====================================================

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # Stratified split to maintain class distribution
)

print("Train-Validation Split:")
print(f"X_train: {X_train.shape}")
print(f"X_val: {X_val.shape}")
print(f"y_train distribution:\n{y_train.value_counts(normalize=True)}")
print(f"\ny_val distribution:\n{y_val.value_counts(normalize=True)}")

# Calculate class weights
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
print(f"\nClass imbalance ratio: 1:{scale_pos_weight:.2f}")

## LightGBM Model Training

In [ ]:
# ====================================================
# STEP 8: Baseline LightGBM Model
# ====================================================

print("="*60)
print("TRAINING BASELINE LightGBM MODEL")
print("="*60)

# Baseline parameters
baseline_params = {
    'num_leaves': 31,
    'max_depth': 8,
    'learning_rate': 0.1,
    'n_estimators': 100,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'scale_pos_weight': scale_pos_weight,
    'is_unbalance': True,
    'objective': 'binary',
    'metric': 'auc',
    'random_state': 42,
    'n_jobs': -1,
    'verbose': -1
}

print("Baseline Parameters:")
for key, value in baseline_params.items():
    print(f"  {key}: {value}")

# Train baseline model
baseline_lgb = LGBMClassifier(**baseline_params)
baseline_lgb.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    callbacks=[
        lgb.log_evaluation(period=20),
        lgb.early_stopping(stopping_rounds=50)
    ]
)

# Baseline predictions
y_val_pred_proba_baseline = baseline_lgb.predict_proba(X_val)[:, 1]
baseline_auc = roc_auc_score(y_val, y_val_pred_proba_baseline)
baseline_ap = average_precision_score(y_val, y_val_pred_proba_baseline)

print(f"\n🎯 Baseline Validation ROC-AUC: {baseline_auc:.4f}")
print(f"🎯 Baseline Validation PR-AUC: {baseline_ap:.4f}")

In [ ]:
# ====================================================
# STEP 9: Hyperparameter Tuning with Optuna
# ====================================================

try:
    import optuna
    print("Optuna version:", optuna.__version__)
    
    def objective(trial):
        params = {
            'num_leaves': trial.suggest_int('num_leaves', 20, 100),
            'max_depth': trial.suggest_int('max_depth', 5, 15),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'subsample': trial.suggest_float('subsample', 0.7, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 30),
            'scale_pos_weight': scale_pos_weight,
            'is_unbalance': True,
            'objective': 'binary',
            'metric': 'auc',
            'random_state': 42,
            'n_jobs': -1,
            'verbose': -1
        }
        
        lgb_model = LGBMClassifier(n_estimators=200, **params)
        
        # Use stratified K-fold cross-validation
        skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
        scores = []
        
        for train_idx, val_idx in skf.split(X_train, y_train):
            X_tr, X_v = X_train.iloc[train_idx], X_train.iloc[val_idx]
            y_tr, y_v = y_train.iloc[train_idx], y_train.iloc[val_idx]
            
            lgb_model.fit(X_tr, y_tr, eval_set=[(X_v, y_v)], callbacks=[lgb.early_stopping(50), lgb.log_evaluation(0)])
            y_pred = lgb_model.predict_proba(X_v)[:, 1]
            auc = roc_auc_score(y_v, y_pred)
            scores.append(auc)
        
        return np.mean(scores)
    
    print("\n" + "="*60)
    print("HYPERPARAMETER TUNING WITH OPTUNA")
    print("="*60)
    
    # Create study
    sampler = optuna.samplers.TPESampler(seed=42)
    study = optuna.create_study(direction='maximize', sampler=sampler)
    
    # Optimize (limited trials for faster execution)
    study.optimize(objective, n_trials=20, show_progress_bar=True)
    
    print("\n" + "="*60)
    print("BEST PARAMETERS FOUND:")
    print("="*60)
    best_params = study.best_params.copy()
    for param, value in best_params.items():
        print(f"  {param}: {value}")
    print(f"\nBest CV AUC: {study.best_value:.4f}")

except ImportError:
    print("⚠️ Optuna not installed. Skipping hyperparameter tuning.")
    print("Install with: pip install optuna")
    best_params = baseline_params.copy()
    best_params.pop('n_estimators')

In [ ]:
# ====================================================
# STEP 10: Train Final LightGBM Model
# ====================================================

print("\n" + "="*60)
print("TRAINING FINAL LightGBM MODEL")
print("="*60)

# Final model parameters
final_params = best_params.copy() if 'best_params' in locals() else baseline_params.copy()
final_params.update({
    'n_estimators': 500,
    'objective': 'binary',
    'metric': 'auc',
    'random_state': 42,
    'n_jobs': -1,
    'verbose': -1
})

# Train final model
final_lgb = LGBMClassifier(**final_params)
final_lgb.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    callbacks=[
        lgb.log_evaluation(period=50),
        lgb.early_stopping(stopping_rounds=50)
    ]
)

# Final predictions
y_val_pred_proba_final = final_lgb.predict_proba(X_val)[:, 1]
y_val_pred_final = (y_val_pred_proba_final > 0.5).astype(int)

final_auc = roc_auc_score(y_val, y_val_pred_proba_final)
final_ap = average_precision_score(y_val, y_val_pred_proba_final)

print(f"\n🎯 Final Model Validation ROC-AUC: {final_auc:.4f}")
print(f"🎯 Final Model Validation PR-AUC: {final_ap:.4f}")
print(f"📈 Improvement over baseline: {(final_auc - baseline_auc) * 100:.2f}%")

In [ ]:
# ====================================================
# STEP 11: Model Evaluation
# ====================================================

print("\n" + "="*60)
print("MODEL EVALUATION METRICS")
print("="*60)

# Classification Report
print("\n📊 CLASSIFICATION REPORT:")
print("-" * 40)
print(classification_report(y_val, y_val_pred_final, target_names=['Non-Fraud', 'Fraud']))

# Confusion Matrix
cm = confusion_matrix(y_val, y_val_pred_final)
print("\n🔢 CONFUSION MATRIX:")
print(cm)

# Additional Metrics
tn, fp, fn, tp = cm.ravel()
print(f"\n📈 ADDITIONAL METRICS:")
print(f"  True Negatives: {tn:,}")
print(f"  False Positives: {fp:,}")
print(f"  False Negatives: {fn:,}")
print(f"  True Positives: {tp:,}")
print(f"\n  Accuracy: {(tp+tn)/(tp+tn+fp+fn):.4f}")
print(f"  Precision (Fraud): {tp/(tp+fp):.4f}")
print(f"  Recall (Fraud): {tp/(tp+fn):.4f}")
print(f"  F1-Score (Fraud): {2*tp/(2*tp+fp+fn):.4f}")
print(f"  ROC-AUC: {final_auc:.4f}")
print(f"  PR-AUC: {final_ap:.4f}")

In [ ]:
# ====================================================
# STEP 12: Visualization - Confusion Matrix & ROC Curves
# ====================================================

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 1. Confusion Matrix Heatmap
sns.heatmap(cm, annot=True, fmt=',d', cmap='Blues', ax=axes[0],
            xticklabels=['Pred: Non-Fraud', 'Pred: Fraud'],
            yticklabels=['Actual: Non-Fraud', 'Actual: Fraud'])
axes[0].set_title('Confusion Matrix', fontsize=14)
axes[0].set_ylabel('Actual')
axes[0].set_xlabel('Predicted')

# 2. ROC Curve
fpr, tpr, thresholds = roc_curve(y_val, y_val_pred_proba_final)
axes[1].plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {final_auc:.4f})')
axes[1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random')
axes[1].fill_between(fpr, tpr, alpha=0.3, color='darkorange')
axes[1].set_xlim([0.0, 1.0])
axes[1].set_ylim([0.0, 1.05])
axes[1].set_xlabel('False Positive Rate', fontsize=12)
axes[1].set_ylabel('True Positive Rate', fontsize=12)
axes[1].set_title('ROC Curve', fontsize=14)
axes[1].legend(loc='lower right')
axes[1].grid(True, alpha=0.3)

# 3. Precision-Recall Curve
precision, recall, _ = precision_recall_curve(y_val, y_val_pred_proba_final)
axes[2].plot(recall, precision, color='green', lw=2, label=f'PR curve (AP = {final_ap:.4f})')
axes[2].fill_between(recall, precision, alpha=0.3, color='green')
axes[2].set_xlim([0.0, 1.0])
axes[2].set_ylim([0.0, 1.05])
axes[2].set_xlabel('Recall', fontsize=12)
axes[2].set_ylabel('Precision', fontsize=12)
axes[2].set_title('Precision-Recall Curve', fontsize=14)
axes[2].legend(loc='lower left')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# ====================================================
# STEP 13: Feature Importance
# ====================================================

# Get feature importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': final_lgb.feature_importances_
}).sort_values('importance', ascending=False)

# Plot top 20 features
plt.figure(figsize=(12, 8))
top_n = 20
sns.barplot(
    x='importance',
    y='feature',
    data=feature_importance.head(top_n),
    palette='viridis'
)
plt.title(f'Top {top_n} Most Important Features (LightGBM)', fontsize=14)
plt.xlabel('Importance Score')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()

print("\n📋 Top 20 Most Important Features:")
print(feature_importance.head(20).to_string(index=False))

In [ ]:
# ====================================================
# STEP 14: Predict on Test Data
# ====================================================

print("="*50)
print("PREDICTING ON TEST DATA")
print("="*50)

# Predict probabilities on test data
test_predictions = final_lgb.predict_proba(X_test_final)[:, 1]

# Create submission DataFrame
submission = pd.DataFrame({
    'TransactionID': test_ids,
    'isFraud': test_predictions
})

print(f"\nSubmission shape: {submission.shape}")
print(f"\nSubmission preview:")
print(submission.head(10))
print(f"\nPrediction statistics:")
print(submission['isFraud'].describe())

# Save submission
submission.to_csv('./submission_lightgbm.csv', index=False)
print("\n✅ Submission saved to 'submission_lightgbm.csv'")

In [ ]:
# ====================================================
# STEP 15: Model Summary
# ====================================================

print("="*60)
print("📊 FINAL LightGBM MODEL SUMMARY")
print("="*60)
print(f"""
Dataset Information:
--------------------
• Training samples: {len(X_train):,}
• Validation samples: {len(X_val):,}
• Test samples: {len(X_test_final):,}
• Number of features: {X_train.shape[1]}

Class Distribution (Training):
------------------------------
• Non-Fraud: {(y_train == 0).sum():,} ({(y_train == 0).mean()*100:.2f}%)
• Fraud: {(y_train == 1).sum():,} ({(y_train == 1).mean()*100:.2f}%)

Model Configuration:
-------------------
• Model Type: LightGBM (Light Gradient Boosting Machine)
• Number of Trees: {final_lgb.n_estimators}
• Max Depth: {final_params.get('max_depth', 'N/A')}
• Learning Rate: {final_params.get('learning_rate', 'N/A')}

Performance Metrics:
-------------------
• ROC-AUC (Baseline): {baseline_auc:.4f}
• ROC-AUC (Final): {final_auc:.4f}
• PR-AUC: {final_ap:.4f}
• Precision (Fraud): {tp/(tp+fp):.4f}
• Recall (Fraud): {tp/(tp+fn):.4f}
• F1-Score (Fraud): {2*tp/(2*tp+fp+fn):.4f}
• Accuracy: {(tp+tn)/(tp+tn+fp+fn):.4f}

Output Files:
-------------
• submission_lightgbm.csv - Test predictions
""")
print("="*60)